<a href="https://colab.research.google.com/github/verneh/dataviz/blob/master/distance_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A simple geospatial attempt.

# Initialize

In [1]:
# import libraries
import requests
from pprint import pprint

In [2]:
# base url for nominatim openstreetmap api
BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'

In [35]:
# grab postcode in the utrecht
postcode = '3511EZ'

response_p = requests.get(f"{BASE_URL}&postalcode={postcode}")
data_p = response_p.json()
pprint(data_p)

[{'boundingbox': ['51.929162885915',
                  '52.249162885915',
                  '4.9522085075117',
                  '5.2722085075117'],
  'class': 'place',
  'display_name': 'Utrecht, Nederland, 3511EZ, Nederland',
  'importance': 0.33499999999999996,
  'lat': '52.08916288591549',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
             'https://osm.org/copyright',
  'lon': '5.112208507511737',
  'place_id': 286220582,
  'type': 'postcode'}]


In [36]:
# extract latitude and longitude
latitude = data_p[0].get('lat')
longitude = data_p[0].get('lon')
print(latitude, longitude)

bbox = data_p[0].get('boundingbox')
print(bbox)

52.08916288591549 5.112208507511737
['51.929162885915', '52.249162885915', '4.9522085075117', '5.2722085075117']


In [43]:
# grab a zipcode nearby. ICC in the Hague. can append &country=italia"
# at the end if you want to be more specific in terms of the country.
# and remove the letters after the number.
zipcode = '2597AK'

response_z = requests.get(f"{BASE_URL}&postalcode={zipcode}")
data_z = response_z.json()

In [44]:
latitude_z = data_z[0].get('lat')
longitude_z = data_z[0].get('lon')

print(latitude_z, longitude_z)

52.109103752941174 4.321444082165395


# Map the points

In [45]:
import folium

# create tuples representing our location
location = float(latitude), float(longitude)
location_z = float(latitude_z), float(longitude_z)

# center the map at Bodegraven
bodegraven = (52.0823, 4.7461)

# create a Folium map centred at the above location
m = folium.Map(location=bodegraven, zoom_start=10, width=800, height=400)

# add markers at the locations
folium.Marker(location, popup="Utrecht").add_to(m)
folium.Marker(location_z, popup="The Hague").add_to(m)

# refer to the map to display it in Jupyter/Colab notebooks
m

In [46]:
# Compute midpoint
midpoint_gen = ((x+y)/2 for x,y in zip(location, location_z))

# convert generator to a tuple representing lat/longitude of the midpoint
midpoint = tuple(midpoint_gen)

print(location)
print(location_z)
print(midpoint)

(52.08916288591549, 5.112208507511737)
(52.109103752941174, 4.321444082165395)
(52.09913331942833, 4.716826294838566)


In [47]:
# create Folium map
m = folium.Map(location=midpoint, zoom_start=10, width=800, height=400)

# add marker at the locations
folium.Marker(location, popup="Start at Utrecht").add_to(m)
folium.Marker(location_z, popup="End at Hague").add_to(m)
folium.Marker(midpoint, popup="Midpoint right here!").add_to(m)

m

# Distance between two points

In [48]:
# Check distance between postcode and zipcode.

from geopy.distance import distance

km = distance(location, location_z)
miles = distance(location, location_z).miles

print("Distance between postcodes:")
print(f"{km}")
print(f"{miles} miles")

Distance between postcodes:
54.23322557317656 km
33.69896403328099 miles


# Draw line between posts.

In [50]:
# Draw lines between posts.

# create a Folium map centred at the above location
m = folium.Map(location=midpoint, zoom_start=10, width=800, height=400)

# add marker at the locations
folium.Marker(location, popup="Our postcode").add_to(m)
folium.Marker(location_z, popup="...and our zipcode.").add_to(m)

# add line between points
folium.PolyLine((location,location_z)).add_to(m)

m